# Transfer auf Embedded System

In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade keras

# Kuzer Zwischentest
Test der Funktion des nicht quantisierten Modells

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
model = load_model('/content/drive/MyDrive/Colab Notebooks/models/modelMitFein_07_08_2024.keras')

In [ ]:
def to_grayscale_with_channels(img_array):
    # Bild in Grayscale konvertieren
    grayscale_image = np.dot(img_array[...,:3], [0.2989, 0.5870, 0.1140])
    # Erzeuge ein 3-Kanal-Grayscale-Bild
    grayscale_image_3_channels = np.stack((grayscale_image,)*3, axis=-1)
    return grayscale_image_3_channels

def load_and_preprocess_image(img_path):
    # Lade das Bild und ändere die Größe
    img = image.load_img(img_path, target_size=(200, 200))
    # Wandle das Bild in ein Array um
    img_array = image.img_to_array(img)
    # Konvertiere das Bild zu Graustufen mit drei Kanälen
    img_array = to_grayscale_with_channels(img_array)
    # Erweitere die Dimensionen, um eine Batch-Dimension hinzuzufügen
    img_array = np.expand_dims(img_array, axis=0)
    # Normalisiere die Pixelwerte (optional, abhängig von der VGG16-Implementierung)
    img_array /= 255.0
    return img_array


In [ ]:
def predict_image(model, img_path, threshold=0.5):
    # Bild laden und vorverarbeiten
    img_array = load_and_preprocess_image(img_path)
    # Vorhersage durchführen
    prediction = model.predict(img_array)
    # Wahrscheinlichkeitswert
    probability = prediction[0][0]
    # Klassenzuordnung basierend auf dem Schwellenwert
    predicted_class = 1 if probability >= threshold else 0
    return probability, predicted_class

img_path = '/content/drive/MyDrive/Colab Notebooks/Set_12_05_24/test/Basilikum/159.jpg'
probability, predicted_class = predict_image(model, img_path)

print(f'Wahrscheinlichkeit für die positive Klasse: {probability}')
print(f'Vorhergesagte Klasse: {predicted_class}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 898ms/step
Wahrscheinlichkeit für die positive Klasse: 3.812175418715924e-05
Vorhergesagte Klasse: 0


In [ ]:
import tensorflow as tf
import keras

In [ ]:
print("Tensorflow Version:", tf.__version__)
print("Keras Version:", keras.__version__)

Tensorflow Version: 2.17.0
Keras Version: 3.4.1


# Neuer versuch Konvertierung ohne Quantisierung

In [ ]:
model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/models/modelMitFein_07_08_2024.keras")

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
new_tf_lite = converter.convert()

Saved artifact at '/tmp/tmp6_spbo8y'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136314482113904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136314398593040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136314398600608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136314398604304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136314398643248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136314398646944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136314398647472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136314398650992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136314398647648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136314398654160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1363143986532

In [ ]:
new_tflite_model_path = '/content/drive/MyDrive/Colab Notebooks/models/newLiteModel_07_08_2024.tflite'

# TensorFlow Lite Modell in eine Datei speichern
with open(new_tflite_model_path, 'wb') as f:
    f.write(new_tf_lite)

# Älterer Versuch mit Quantisierung

In [ ]:
# Konvertierung des trainierten Modells in ein Tensorflow Lite Modell
converter = tf.lite.TFLiteConverter.from_saved_model("/content/drive/MyDrive/Colab Notebooks/models/export")
converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [ ]:
# Quantisierungseinstellungen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Funktion zur Erweiterung des Graustufenbildes auf drei Kanäle
def to_grayscale_with_channels(image):
    # Bild in Grayscale konvertieren
    grayscale_image = np.dot(image[...,:3], [0.2989, 0.5870, 0.1140])

    # Erzeuge ein 3-Kanal-Grayscale-Bild
    grayscale_image_3_channels = np.stack((grayscale_image,)*3, axis=-1)

    return grayscale_image_3_channels

test_dir = "/content/drive/MyDrive/Colab Notebooks/Set_12_05_24/test"

test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=to_grayscale_with_channels,
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(200, 200),
    batch_size=89,
    class_mode="binary",
    color_mode="rgb"
)

Found 89 images belonging to 2 classes.


In [ ]:
# tensorflow lite benötigt ein representatives Dataset, um die Quantisierung durchführen zu können
# es werden hierfür die Testdaten verwendet
def representative_dataset():
  for _ in range(len(test_generator)):
    batch = next(test_generator)[0]
    yield [batch]

In [ ]:
# Setze das repräsentative Datenset für die Quantisierung
converter.representative_dataset = representative_dataset

In [ ]:
# Konvertierung ausführen
tflite_model = converter.convert()

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [ ]:
tflite_model_path = '/content/drive/MyDrive/Colab Notebooks/models/liteModel.tflite'

# TensorFlow Lite Modell in eine Datei speichern
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

In [ ]:
# import tensorflow as tf

# Laden des TensorFlow Lite Modells
interpreter = tf.lite.Interpreter(model_path="/content/drive/MyDrive/Colab Notebooks/models/liteModel.tflite")
interpreter.allocate_tensors()

In [ ]:
import os

# Details des Eingabentensors
input_details = interpreter.get_input_details()
print("Eingabetensor Details:", input_details)

# Details des Ausgabetensors
output_details = interpreter.get_output_details()
print("Ausgabetensor Details:", output_details)

# Größe des Modells
model_size = os.path.getsize("/content/drive/MyDrive/Colab Notebooks/models/liteModel.tflite")
print("Modellgröße:", model_size, "Bytes")

Eingabetensor Details: [{'name': 'serving_default_input_layer_1:0', 'index': 0, 'shape': array([  1, 200, 200,   3], dtype=int32), 'shape_signature': array([ -1, 200, 200,   3], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.003921176306903362, 0), 'quantization_parameters': {'scales': array([0.00392118], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Ausgabetensor Details: [{'name': 'StatefulPartitionedCall_1:0', 'index': 55, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.00390625, 0), 'quantization_parameters': {'scales': array([0.00390625], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Modellgröße: 19559528 Bytes


# Test des TF-Lite Modells

In [ ]:
import tensorflow as tf

In [ ]:
print("Tensorflow Version:", tf.__version__)

Tensorflow Version: 2.17.0


## TF-Lite Modell laden

In [ ]:
interpreter = tf.lite.Interpreter(model_path="/content/drive/MyDrive/Colab Notebooks/models/newLiteModel_07_08_2024.tflite")

## Eingabe und Ausgabe Details des Modells abrufen

In [ ]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [  1 200 200   3]
Input Type: <class 'numpy.float32'>
Output Shape: [1 1]
Output Type: <class 'numpy.float32'>


## Resize Tensors
89 Test images --> Input Tensor soll 89 Bilder gleichzeitig bekommen.

In [ ]:
interpreter.resize_tensor_input(input_details[0]['index'], (89, 200, 200, 3))
interpreter.resize_tensor_input(output_details[0]['index'], (89, 1))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])


Input Shape: [ 89 200 200   3]
Input Type: <class 'numpy.float32'>
Output Shape: [89  1]
Output Type: <class 'numpy.float32'>


## Datenvorbereitung

In [ ]:
x_batch, y_batch = next(test_generator)
y_batch = y_batch.reshape(-1, 1)

print("Test Images dtype:", x_batch.dtype, "Test Images shape:", x_batch.shape)
print("Test Labels dtype:", y_batch.dtype, "Test Labels shape:", y_batch.shape)

Test Images dtype: float32 Test Images shape: (1, 200, 200, 3)
Test Labels dtype: float32 Test Labels shape: (1, 1)


In [ ]:
x_batch_uint8 = (x_batch * 255).astype(np.uint8)
y_batch_uint8 = y_batch.astype(np.uint8)

print("Test Images dtype:", x_batch_uint8.dtype, "Test Images shape:", x_batch_uint8.shape)
print("Test Labels dtype:", y_batch_uint8.dtype, "Test Labels shape:", y_batch_uint8.shape)

Test Images dtype: uint8 Test Images shape: (89, 200, 200, 3)
Test Labels dtype: uint8 Test Labels shape: (89,)


## Inferenz

In [ ]:
interpreter.set_tensor(input_details[0]['index'], x_batch)
interpreter.invoke()

tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmax(tflite_model_predictions, axis=1)

Prediction results shape: (89, 1)


## Genauigkeit berechnen

In [ ]:
accuracy = np.mean(prediction_classes == y_batch)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 53.93%


Die Genauigkeit des Modells ist durch die Konvertierung in Tensorflow Lite deutlich abgefallen und somit ist das Modell so nicht nutzbar.

Es wurde sich dazu entschieden das ursprüngliche Modell auf einem RaspberryPi zu verwenden ohne es zu konvertieren.

## Inferenz durchführen

In [ ]:
import tensorflow as tf
import numpy as np

class TFLiteModel:
    def __init__(self, model_path: str):
        self.interpreter = tf.lite.Interpreter(model_path=model_path)
        self.interpreter.allocate_tensors()

        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()
        self.input_scale, self.input_zero_point = self.input_details[0]['quantization']
        self.output_scale, self.output_zero_point = self.output_details[0]['quantization']

    def predict(self, *data_args):
        assert len(data_args) == len(self.input_details)
        for data, details in zip(data_args, self.input_details):
            self.interpreter.set_tensor(details["index"], data)
        self.interpreter.invoke()
        output_data = self.interpreter.get_tensor(self.output_details[0]["index"])
        return output_data

    def predict_single_image(self, generator):
        # Nimm nur das erste Batch aus dem Generator
        data, labels = next(generator)

        # Nimm nur das erste Bild und Label aus dem Batch
        single_data = np.expand_dims(data[0], axis=0).astype(np.uint8)
        true_label = labels[0]

        # Vorhersage
        pred = self.predict(single_data)[0]
        decoded_pred = self.output_scale * (pred - self.output_zero_point)

        # Print das wahre Label und die Vorhersage
        print("True Label:", true_label)
        print("Model Prediction (Raw):", decoded_pred)
        predicted_label = (decoded_pred > 0.5).astype(int).flatten()[0]
        print("Predicted Label:", predicted_label)

        return predicted_label, true_label

# Lade das TFLite-Modell
model = TFLiteModel("/content/drive/MyDrive/Colab Notebooks/models/liteModel.tflite")

# Vorhersage für ein einzelnes Bild
predicted_label, true_label = model.predict_single_image(test_generator)

# Berechne die Genauigkeit (für nur ein Bild)
accuracy = (predicted_label == true_label).astype(int)
print("Accuracy:", accuracy)


True Label: 1.0
Model Prediction (Raw): [0.98046875]
Predicted Label: 1
Accuracy: 1
